# Imports

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt


# Data source

In [ ]:
df = pd.read_csv('raw_json_structure.csv')


# Data cleaning

In [ ]:
df.fillna(0, inplace = True)
df.replace('None', 0, inplace = True)


# EDA

In [ ]:
# some basic stats

documents = df['id'].unique()
departments = df['task_responsibles_groups'].unique()
employees = df['name'].unique()
positions = df['position'].unique()


print('Number of docs:', len(documents))
print('Number of depts in the company:', len(departments) )
print('Number of employees in the company:', len(employees))
print('Number of positions in the company total:', len(positions))

## How many documents was analyzed?

In [ ]:
print(docs)

## Company departments

In [ ]:
print(departments)

## Employees

In [ ]:
print(employees)

# Graph

In [ ]:
DG = nx.DiGraph()

comp_name = 'URFU_COMP'

for i in list(departments):
    


DG.add_edge('ООО "Мегасофт"', departments_Unique_list[0]) 
DG.add_edge('ООО "Мегасофт"', departments_Unique_list[1]) 
DG.add_edge('ООО "Мегасофт"', departments_Unique_list[2]) 
DG.add_edge('ООО "Мегасофт"', departments_Unique_list[3]) 
DG.add_edge('ООО "Мегасофт"', departments_Unique_list[4]) 
DG.add_edge('ООО "Мегасофт"', departments_Unique_list[5]) 
DG.add_edge('ООО "Мегасофт"', departments_Unique_list[6]) 
DG.add_edge('ООО "Мегасофт"', departments_Unique_list[7]) 

nx.draw(DG, with_labels=True, font_weight='bold')
